In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import tensorflow as tf
tf.keras.backend.clear_session()
import pickle 

In [2]:
df = pd.read_csv("data/fire_alarm.csv")
X = df.drop('Fire Alarm', axis=1)
y = df['Fire Alarm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
knn_model = KNeighborsClassifier(3)
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [9]:
with open('knn.pkl', 'wb') as pickle_out:
    pickle.dump(knn_model, pickle_out)

In [10]:
with open('knn.pkl', 'rb') as pickle_in: 
    unpickled_list = pickle.load(pickle_in)

KNeighborsClassifier(n_neighbors=3)


In [7]:
rf = RandomForestClassifier(random_state=42).fit(X_train, y_train)

In [11]:
with open('rf.pkl', 'wb') as pickle_out:
    pickle.dump(rf, pickle_out)

In [12]:
base_classifiers = [
    ('logistic', LogisticRegression()),
    ('random_forest', RandomForestClassifier(n_estimators=100)),
    ('knn', KNeighborsClassifier(3))
]

meta_classifier = DecisionTreeClassifier()
stacking_classifier = StackingClassifier(estimators=base_classifiers, final_estimator=meta_classifier)
stacking_classifier.fit(X_train, y_train)

C:\Users\Lostg\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Lostg\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

StackingClassifier(estimators=[('logistic', LogisticRegression()),
                               ('random_forest', RandomForestClassifier()),
                               ('knn', KNeighborsClassifier(n_neighbors=3))],
                   final_estimator=DecisionTreeClassifier())

In [13]:
with open('stacking_classifier.pkl', 'wb') as pickle_out:
    pickle.dump(stacking_classifier, pickle_out)

In [14]:
base_classifier = DecisionTreeClassifier()
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=100, random_state=42)
bagging_classifier.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100,
                  random_state=42)

In [15]:
with open('bagging_classifier.pkl', 'wb') as pickle_out:
    pickle.dump(bagging_classifier, pickle_out)

In [16]:
from sklearn.cluster import KMeans

In [17]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
kmeans.fit(X_train)
predictions = kmeans.predict(X_test)

In [18]:
with open('kmeans.pkl', 'wb') as pickle_out:
    pickle.dump(kmeans, pickle_out)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X[:30000], y[:30000], test_size=0.2, random_state=42)

In [18]:
model_cl = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(32, activation="relu", input_shape=(7,)),
        tf.keras.layers.Dense(16, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)

In [19]:
model_cl.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss="sparse_categorical_crossentropy")

In [20]:
model_cl.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 32)                256       
                                                                 
 dense_13 (Dense)            (None, 16)                528       
                                                                 
 dropout_6 (Dropout)         (None, 16)                0         
                                                                 
 dense_14 (Dense)            (None, 8)                 136       
                                                                 
 dropout_7 (Dropout)         (None, 8)                 0         
                                                                 
 dense_15 (Dense)            (None, 2)                 18        
                                                                 
Total params: 938 (3.66 KB)
Trainable params: 938 (3.6

In [21]:
model_cl.fit(X_train, y_train, epochs=30, batch_size=64)

Epoch 1/30


UnimplementedError: Graph execution error:

Detected at node sparse_categorical_crossentropy/Cast defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Lostg\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Lostg\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\Lostg\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\Lostg\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\Lostg\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\Lostg\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\Lostg\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\Lostg\anaconda3\Lib\site-packages\ipykernel\kernelbase.py", line 516, in dispatch_q

In [ ]:
y_pred_prob = model_cl.predict(X_classification_test)
y_pred = tf.argmax(y_pred_prob, axis=1)